# Hyperdrive Contract Integration

In [18]:
from ape import accounts, chain, networks, Project
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

## Compilation and Network Setup

In [19]:
networks.parse_network_choice('ethereum:local:foundry').__enter__()
project_root = Path.cwd().parent.parent
project = Project(path=project_root)

INFO: Connecting to existing 'anvil' process.


## Deploy Config

In [20]:
# deployment config values

base_supply = int(1e24) # 100 million
t_stretch = int(22.186877016851916266e18)
initial_rate = int(5e18) # share rate
initial_apr = int(.05e18)
share_price = int(1e18)
checkpoints = int(365)
checkpoint_duration = int(86400)
curve_fee = int(0)
flat_fee = int(0)

## Deploy Contracts

In [21]:
# generate deployer account
deployer = accounts.test_accounts.generate_test_account()

# give deployer 3 eth
deployer.balance += int(3e18)

base_address = deployer.deploy(project.ERC20Mintable)
base_ERC20 = project.ERC20Mintable.at(base_address)


fixed_math_address = deployer.deploy(project.MockFixedPointMath)
fixed_math = project.MockFixedPointMath.at(fixed_math_address)

base_ERC20.mint(base_supply, sender=deployer)
time_stretch = fixed_math.divDown(int(1e18), t_stretch)

hyperdrive_address = deployer.deploy(project.MockHyperdriveTestnet, base_ERC20,
                                     initial_rate, share_price, checkpoints,
                                     checkpoint_duration, int(time_stretch), curve_fee,
                                     flat_fee)
hyperdrive = project.MockHyperdriveTestnet.at(hyperdrive_address)

with accounts.use_sender(deployer):
    base_ERC20.approve(hyperdrive, base_supply)
    hyperdrive.initialize(base_supply, initial_rate, deployer, False)


INFO: Confirmed 0x5a913ce499b3b4293306ab705c245f035ec04764d1dff2894e37048234a1f540 (total fees paid = 2095680000000000)
SUCCESS: Contract 'ERC20Mintable' deployed to: 0x2cfF6c349E591F7D0F312bCBF9e07E6d51953561
INFO: Confirmed 0x3d97eef5898c15abbdb79703ed7372ea18367e6d3744ce9ac97be7ec80dceb60 (total fees paid = 1641477591436735)
SUCCESS: Contract 'MockFixedPointMath' deployed to: 0xCC9c7ddCCfa265A9561cbf8Fb1e36e48792671fe
INFO: Confirmed 0x94252dc80de6d586515c8c4bd99a18101022e9f3f0b826d49c42a3a8819f801e (total fees paid = 188132742201476)
INFO: Confirmed 0x302f4b44fada0134b1da8079344bb0cda0d6ccc2e85d1d4849baa296998563b5 (total fees paid = 13465444649738330)
SUCCESS: Contract 'MockHyperdriveTestnet' deployed to: 0x0A2E4E22537bf48e89ae566F582A7ee3Dd19B251
INFO: Confirmed 0x3916fc51695b9983a623e7cfbeee7b64dc9c6df0939821e96e78d5986ac304e3 (total fees paid = 121332006524121)
INFO: Confirmed 0xa37ee29480b18d488a0d9ec440009d1aedccd0f127c6d3456f028b03178885ea (total fees paid = 537323978979645)

## User Config

In [22]:
# User Config Values

user_balance = int(1e24)
long_amount = int(1e21) # 1000 base amount
min_out = int(1e21)
num_longs = 50

In [26]:
# Generate hyperdrive user
hyperdrive_user = accounts.test_accounts.generate_test_account()

# Give user 1 eth balance
hyperdrive_user.balance += int(1e18)

# get current block
current_block = chain.blocks[-1].number

pool_info = [hyperdrive.getPoolInfo().__dict__]
pool_info[0]["block_number_"] = current_block

def open_longs():
    # with notation avoids passing in sender each time
    with accounts.use_sender(hyperdrive_user):
        # Mint 1 million DAI
        base_ERC20.mint(user_balance)

        # Approve ERC20 usage by contract
        base_ERC20.approve(hyperdrive, user_balance)

        pool_info = hyperdrive.getPoolInfo().__dict__

        # Open Long on 10,000 DAI
        tx_receipt = hyperdrive.openLong(long_amount, min_out, hyperdrive_user, False)
        pt_balance = tx_receipt.return_value
        
        print("user input %s base" %(long_amount))
        print("user received %s PTs" %(pt_balance))
        
        current_block = chain.blocks[-1]
        pool_info["block_number_"] = current_block.number

        return pool_info


for x in range(num_longs):        
    pool_info += [open_longs()]


INFO: Confirmed 0x6b3f8ec907a9299738a6642b3751909e8b2fed57ebc137bbff27139c361505b6 (total fees paid = 108712252613644)
INFO: Confirmed 0x7030bba00fe631196b37de0050c3a974932cffff9198f743c388cba69b72b26c (total fees paid = 98391790111901)
hello


ContractLogicError: Transaction failed.

In [ ]:
df = pd.DataFrame(pool_info)

plt.figure(1)
plt.plot(df['block_number_'], df['shareReserves_'], label='Share Reserves')
plt.plot(df['block_number_'], df['bondReserves_'], label='Bond Reserves')
plt.plot(df['block_number_'], df['lpTotalSupply'], label='LP Supply')
plt.xlabel('Block Number')
plt.ylabel('Reserves & Supply')
plt.title('Local Longs Opened Consecutively')
plt.legend()
plt.show()